In [1]:
import json
import cv2
import matplotlib.pyplot as plt
import os

In [3]:
data = json.load(open("./data/video2_data/instances_default.json"))

In [4]:
data["categories"]

[{'id': 1, 'name': 'Auto Privado', 'supercategory': ''},
 {'id': 2, 'name': 'Bus', 'supercategory': ''},
 {'id': 3, 'name': 'Camioneta rural', 'supercategory': ''},
 {'id': 4, 'name': 'Microbus', 'supercategory': ''},
 {'id': 5, 'name': 'Mototaxi', 'supercategory': ''},
 {'id': 6, 'name': 'Moto lineal', 'supercategory': ''},
 {'id': 7, 'name': 'Omnibus Interprovincial', 'supercategory': ''},
 {'id': 8, 'name': 'Auto colectivo', 'supercategory': ''},
 {'id': 9, 'name': 'Articulado', 'supercategory': ''}]

In [5]:
print(len(data["images"]))
print(len(data["annotations"]))

29290
7421


In [6]:
data["annotations"][0]

{'id': 1,
 'image_id': 1,
 'category_id': 5,
 'segmentation': [],
 'area': 6938.2555000000075,
 'bbox': [1603.3, 281.79, 75.13, 92.35],
 'iscrowd': 0,
 'attributes': {'occluded': False, 'rotation': 0.0}}

In [7]:
img_with_annotation = [i["image_id"] for i in data["annotations"]]

# Redimention and save img

In [9]:
cap = cv2.VideoCapture("./data/videos/video2.mp4")
_,frame = cap.read()
r = frame.shape[0]/frame.shape[1] # H/W
new_W = frame.shape[1]/5
dim = (int(new_W), int(r * new_W))

H,W= dim
dim

(384, 216)

In [10]:
cap = cv2.VideoCapture("./data/videos/video2.mp4")
count = 1
set_img = set(img_with_annotation)
success, frame = cap.read()
while success:
    name = "./data/videos/dummi2/frame_" + str(count) + ".png"
    if count in set_img:
        resized = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
        cv2.imwrite(name,resized)  # SAVE IMG
    success, frame = cap.read()
    count = count + 1
cap.release()

In [13]:
Ho = 1080
Wo = 1920
W

216

In [11]:
def coco_to_yolo(coco_bbox, image_width=W, image_height=H):
    x_min, y_min, width, height = coco_bbox
    
    # COCO format to YOLO format
    center_x = (x_min + width / 2) / image_width
    center_y = (y_min + height / 2) / image_height
    yolo_width = width / image_width
    yolo_height = height / image_height
    
    return [center_x, center_y, yolo_width, yolo_height]

def yolo_to_coco(yolo_bbox, image_width=W, image_height=H):
    center_x, center_y, yolo_width, yolo_height = yolo_bbox
    
    # YOLO format to COCO format
    x_min = int((center_x - yolo_width / 2) * image_width)
    y_min = int((center_y - yolo_height / 2) * image_height)
    width = int(yolo_width * image_width)
    height = int(yolo_height * image_height)
    
    return [x_min, y_min, width, height]

In [19]:
for i in data["annotations"]:
    f = open("./data/videos/dummi2_labels/frame_"+str(i["image_id"])+".txt", "a")
    f.write(str(int(i["category_id"]-1))+ " ")
    
    # cl,x,y,w,h = [eval(i) for i in boxx] YOLO
    # # Bboxes are in [top-left-x, top-left-y, width, height]  COCO
    box = coco_to_yolo(i["bbox"])

    # due of resized
    box[0] = box[0]*W/Wo
    box[1] = box[1]*H/Ho
    box[2] = box[2]*W/Wo
    box[3] = box[3]*H/Ho
    
    for b in box:
        f.write( f"{b:.5f}" + " ")
    f.write("\n")
    f.close()

In [15]:
def oslistdir(directory):
    def get_creation_time(entry):
        return entry.stat().st_ctime

    with os.scandir(directory) as entries:
        sorted_entries = sorted(entries, key=get_creation_time)
        sorted_items = [entry.name for entry in sorted_entries]
    return sorted_items

In [20]:
names = []
for i in oslistdir("./data/videos/dummi2"):
    i = i.split(".")[0]
    names.append(i)

In [44]:
# CREATE VIDEO WITH IMG WITH ANNOTATIONS
img_array = []
for name in names:
    img = cv2.imread("./data/videos/dummi2/" + name + ".png")
    box=[]
    with open("./data/videos/dummi2_labels/" + name + ".txt", "r") as f:
        for i in f:
            box.append(i.split())
    f.close()

    for boxx in box:
    
        cl,x,y,w,h = [eval(i) for i in boxx]

        h_frame,w_frame,channel = frame.shape

        # top left
        xtl = int(x*w_frame-w*w_frame/2)
        ytl = int(y*h_frame+h*h_frame/2)

        # bottom right
        xbr = int(x*w_frame+w*w_frame/2)
        ybr = int(y*h_frame-h*h_frame/2)

        img = cv2.rectangle(img, (xtl,ytl),(xbr,ybr),(0,255,0),2 )


    height, width, layers = frame.shape
    size = (width, height)
    img_array.append(img)

out = cv2.VideoWriter("./data/videos/video2_low.mp4",cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 2, size)


for img in img_array:
    #time.sleep()
    out.write(img)
out.release()

In [37]:
print(frame.shape)
print(size)

(216, 384, 3)
(384, 216)


In [25]:
frame.shape

(216, 384, 3)